In [2]:
import pyspark 
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

22/03/01 15:11:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_green.registerTempTable('green')

In [6]:
from pyspark.sql import types

In [7]:
df_green = df_green.withColumn("lpep_pickup_datetime", df_green["lpep_pickup_datetime"].cast(types.TimestampType()))

In [8]:
df_green_revenue = spark.sql("""
SELECT 
    ---Revenue grouping
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour_month,
    
    SUM(total_amount) AS amount_,
    COUNT(1) AS number_records
FROM green
GROUP BY 1,2

""")

In [9]:
df_green_revenue.show()

+----+-------------------+------------------+--------------+
|zone|         hour_month|           amount_|number_records|
+----+-------------------+------------------+--------------+
| 129|2020-01-19 17:00:00|117.61999999999998|            10|
|  33|2020-01-10 19:00:00|            456.18|            28|
|  74|2020-01-15 02:00:00|             73.81|             6|
| 166|2020-01-23 08:00:00|            544.23|            33|
|  83|2020-01-04 02:00:00|56.260000000000005|             5|
| 134|2020-01-15 08:00:00| 55.85999999999999|             4|
|  81|2020-01-27 18:00:00|             90.86|             3|
| 223|2020-01-13 09:00:00|             70.41|             5|
| 196|2020-01-29 15:00:00|            289.47|            15|
| 247|2020-01-30 15:00:00|43.900000000000006|             3|
| 213|2020-01-05 09:00:00|206.48999999999998|             7|
|  24|2020-01-07 05:00:00|              63.5|             4|
| 244|2020-01-26 06:00:00|13.600000000000001|             2|
| 244|2020-01-03 14:00:0

In [9]:
df_green_revenue.write.parquet('data/report/revenue/green')

In [10]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [11]:
df_yellow_revenue = spark.sql("""
SELECT 
    ---Revenue grouping
    PULocationID AS zone,
    date_trunc('hour', tpep_pickup_datetime) AS hour_month,
    
    SUM(total_amount) AS amount_,
    COUNT(1) AS number_records
FROM yellow
WHERE tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1,2

""")

In [12]:
df_yellow_revenue.show()

+----+-------------------+------------------+--------------+
|zone|         hour_month|           amount_|number_records|
+----+-------------------+------------------+--------------+
|  43|2020-01-24 20:00:00|2786.8399999999974|           180|
| 230|2020-01-29 18:00:00| 11976.85000000003|           664|
| 163|2020-01-14 21:00:00| 6199.080000000006|           381|
|  79|2020-01-01 00:00:00|12573.810000000034|           721|
| 132|2020-01-12 20:00:00|31807.889999999992|           590|
|  43|2020-01-24 08:00:00| 3534.329999999997|           232|
| 162|2020-01-26 13:00:00| 4528.120000000001|           295|
| 238|2020-01-08 08:00:00|5751.6500000000015|           348|
| 249|2020-01-28 08:00:00| 2671.499999999998|           179|
|  68|2020-01-09 23:00:00| 4352.069999999998|           231|
| 263|2020-01-23 13:00:00| 2498.149999999998|           188|
| 234|2020-01-15 23:00:00| 5139.520000000002|           308|
|  90|2020-01-04 10:00:00| 2216.149999999998|           167|
| 262|2020-01-15 11:00:0

In [13]:
df_yellow_revenue.write.parquet('data/report/revenue/yellow')